# Supermarket Sales Data Basic ETL and Pipeline Creation
##### by Harmon Tuazon

Tool/Language Stack:
1. Python
2. SQL (PostgreSQL)
3. PowerBI 

The following attributes/columns needed cleaning due to formatting issues:

1. Invoice ID
2. Tax 5%
3. Total
4. Date
5. Cost of Goods Sold (COGs)

In [50]:
#Importing the necessary packages for file manipulation and data cleaning
import pandas as pd
from sqlalchemy import create_engine 
from dotenv import find_dotenv, load_dotenv
import os


First we will open the file in the CSV and preview how the dataset looks like


In [51]:
#Finding env file so that we can use env variables
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [52]:
#Declaring the file path of the CSV
INFILE = os.getenv("INFILE").strip("r").strip('"')

#Converting csv to dataframe
sales_df = pd.read_csv(INFILE, index_col="Invoice ID")    
sales_df

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Date,Time,Payment,cogs,Rating
Invoice ID,,,,,,,,,,,,
750-67-8428,A,Yangon,Member,Female,Health and beauty,78.4245,7,1/5/2019,13:08,Ewallet,522.83,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,16.0440,5,3/8/2019,10:29,Cash,76.40,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,48.6465,7,3/3/2019,13:23,Credit card,324.31,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,61.1310,8,1/27/2019,20:33,Ewallet,465.76,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,90.6255,7,2/8/2019,10:37,Ewallet,604.17,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...
233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,42.3675,1,1/29/2019,13:46,Ewallet,40.35,6.2
303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,102.2490,10,3/2/2019,17:16,Ewallet,973.80,4.4
727-02-1313,A,Yangon,Member,Male,Food and beverages,33.4320,1,2/9/2019,13:22,Cash,31.84,7.7


In [53]:
#Checking the datatypes of each column to see if any changes need to be made
sales_df.dtypes

Branch            object
City              object
Customer type     object
Gender            object
Product line      object
Unit price       float64
Quantity           int64
Date              object
Time              object
Payment           object
cogs             float64
Rating           float64
dtype: object

In [54]:
#First I will remove the "-" (dash) character in the index 
sales_df.index = sales_df.index.str.replace('-', '', regex=False)

#Second I will round Unit price to 2 decimal places to match currency format
sales_df["Unit price"] = sales_df["Unit price"].round(2)

#Third I will convert columns into flags 
sales_df["Customer type"] = sales_df["Customer type"].apply(lambda x: 1 if x == "Member" else 0)

#printing to see new format
sales_df



,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Date,Time,Payment,cogs,Rating
Invoice ID,,,,,,,,,,,,
750678428,A,Yangon,1,Female,Health and beauty,78.42,7,1/5/2019,13:08,Ewallet,522.83,9.1
226313081,C,Naypyitaw,0,Female,Electronic accessories,16.04,5,3/8/2019,10:29,Cash,76.40,9.6
631413108,A,Yangon,0,Male,Home and lifestyle,48.65,7,3/3/2019,13:23,Credit card,324.31,7.4
123191176,A,Yangon,1,Male,Health and beauty,61.13,8,1/27/2019,20:33,Ewallet,465.76,8.4
373737910,A,Yangon,0,Male,Sports and travel,90.63,7,2/8/2019,10:37,Ewallet,604.17,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...
233675758,C,Naypyitaw,0,Male,Health and beauty,42.37,1,1/29/2019,13:46,Ewallet,40.35,6.2
303962227,B,Mandalay,0,Female,Home and lifestyle,102.25,10,3/2/2019,17:16,Ewallet,973.80,4.4
727021313,A,Yangon,1,Male,Food and beverages,33.43,1,2/9/2019,13:22,Cash,31.84,7.7


In [55]:
#add total column
sales_df["Total Sales"] = sales_df["Unit price"] * sales_df["Quantity"]
#add gross margin
sales_df["Gross Margin"] = sales_df["Total Sales"] - sales_df["cogs"]

#add cogs per unit and gross margin per unit
sales_df["COGS per Unit"] = sales_df["cogs"] / sales_df["Quantity"]
sales_df["Gross Margin per Unit"] = sales_df["Gross Margin"] / sales_df["Quantity"]


#dropping  time as it is not necessary for this analysis
sales_df.drop("Time", axis= 1, inplace=True)



Next, we will begin loading this to our PostgreSQL database for storage and more efficient querying

In [56]:
#Splitting the dataframe into two dataframes just to help in practicing queries and make each table/entity more specific
#Resetting index since I need to split the columns with Invoice ID as a column 
sales_df = sales_df.reset_index()

#Declaring my new dataframes
customerProfile_df = sales_df[['Invoice ID', 'Branch', 'City', 'Gender', 'Payment', 'Date']]
revenue_df = sales_df[['Invoice ID', 'Quantity', 'Total Sales', 'Gross Margin']]
productDetails_df = sales_df[['Invoice ID', 'Product line', 'Unit price', 'COGS per Unit', 'Gross Margin per Unit', 'Rating']]


#Set invoice id as index for the new dataframes
customerProfile_df = customerProfile_df.set_index('Invoice ID')
revenue_df = revenue_df.set_index('Invoice ID')
productDetails_df = productDetails_df.set_index('Invoice ID')





In [57]:
customerProfile_df

,Branch,City,Gender,Payment,Date
Invoice ID,,,,,
750678428,A,Yangon,Female,Ewallet,1/5/2019
226313081,C,Naypyitaw,Female,Cash,3/8/2019
631413108,A,Yangon,Male,Credit card,3/3/2019
123191176,A,Yangon,Male,Ewallet,1/27/2019
373737910,A,Yangon,Male,Ewallet,2/8/2019
...,...,...,...,...,...
233675758,C,Naypyitaw,Male,Ewallet,1/29/2019
303962227,B,Mandalay,Female,Ewallet,3/2/2019
727021313,A,Yangon,Male,Cash,2/9/2019


In [58]:
revenue_df

,Quantity,Total Sales,Gross Margin
Invoice ID,,,
750678428,7,548.94,26.11
226313081,5,80.20,3.80
631413108,7,340.55,16.24
123191176,8,489.04,23.28
373737910,7,634.41,30.24
...,...,...,...
233675758,1,42.37,2.02
303962227,10,1022.50,48.70
727021313,1,33.43,1.59


In [59]:
productDetails_df

,Product line,Unit price,COGS per Unit,Gross Margin per Unit,Rating
Invoice ID,,,,,
750678428,Health and beauty,78.42,74.69,3.73,9.1
226313081,Electronic accessories,16.04,15.28,0.76,9.6
631413108,Home and lifestyle,48.65,46.33,2.32,7.4
123191176,Health and beauty,61.13,58.22,2.91,8.4
373737910,Sports and travel,90.63,86.31,4.32,5.3
...,...,...,...,...,...
233675758,Health and beauty,42.37,40.35,2.02,6.2
303962227,Home and lifestyle,102.25,97.38,4.87,4.4
727021313,Food and beverages,33.43,31.84,1.59,7.7


In [60]:
# Define your PostgreSQL credentials
username = os.getenv("DB_USER")
password = os.getenv("DB_PW")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_DATABASE")



In [61]:
# Create the SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

# Write DataFrame to PostgreSQL table
customerProfile_df.to_sql('customer_profile', engine, if_exists='replace', index=True, index_label="invoice_id")
revenue_df.to_sql('revenue', engine, if_exists='replace', index=True, index_label="invoice_id")
productDetails_df.to_sql('product_details', engine, if_exists='replace', index=True, index_label="invoice_id")

1000